In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
raw_customer_csv = pd.read_csv(
    'data/olist_customers_dataset.csv')
raw_geoloc_csv = pd.read_csv('data/olist_geolocation_dataset.csv')
raw_order_items = pd.read_csv('data/olist_order_items_dataset.csv')
raw_order = pd.read_csv('data/olist_orders_dataset.csv')
raw_product = pd.read_csv('data/olist_products_dataset.csv')
raw_payment = pd.read_csv('data/olist_order_payments_dataset.csv')
raw_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
raw_sellers = pd.read_csv('data/olist_sellers_dataset.csv')
raw_category = pd.read_csv('data/product_category_name_translation.csv')

In [3]:
raw_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


# Cleaning data

## Product

In [4]:
products = pd.merge(raw_product, raw_category,
                    how="left",
                    on="product_category_name")

# We focus on client so product column is not relevant
del_features_list = ["product_category_name", "product_weight_g",
                     "product_length_cm", "product_height_cm",
                     "product_width_cm"]

products.drop(del_features_list, axis=1, inplace=True)

products = products.rename(columns={"product_category_name_english":
                                    "product_category_name"})

## Command

In [5]:
order_items = pd.merge(raw_order_items, raw_order,
                       how="left",
                       on="order_id")

del_features_list = ["seller_id", "shipping_limit_date",
                     "order_approved_at",
                     "order_delivered_carrier_date",
                     "order_estimated_delivery_date"]

order_items.drop(del_features_list,
                 axis=1,
                 inplace=True)

# Modify object cols in timestamp format
datetime_cols = ["order_purchase_timestamp", "order_delivered_customer_date"]
for col in datetime_cols:
    order_items[col] = order_items[col].astype('datetime64[ns]')
order_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       112650 non-null  object        
 1   order_item_id                  112650 non-null  int64         
 2   product_id                     112650 non-null  object        
 3   price                          112650 non-null  float64       
 4   freight_value                  112650 non-null  float64       
 5   customer_id                    112650 non-null  object        
 6   order_status                   112650 non-null  object        
 7   order_purchase_timestamp       112650 non-null  datetime64[ns]
 8   order_delivered_customer_date  110196 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(4)
memory usage: 8.6+ MB


### Payments

In [6]:
group_payments = raw_payment.groupby(by="order_id").agg(
    {"payment_sequential": 'count',
     "payment_installments": 'sum'})

order_items = pd.merge(order_items, group_payments,
                       how="left",
                       on="order_id")

order_items = order_items.rename(columns={
    "payment_sequential": "nb_payment_sequential",
    "payment_installments": "sum_payment_installments"})

order_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 11 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       112650 non-null  object        
 1   order_item_id                  112650 non-null  int64         
 2   product_id                     112650 non-null  object        
 3   price                          112650 non-null  float64       
 4   freight_value                  112650 non-null  float64       
 5   customer_id                    112650 non-null  object        
 6   order_status                   112650 non-null  object        
 7   order_purchase_timestamp       112650 non-null  datetime64[ns]
 8   order_delivered_customer_date  110196 non-null  datetime64[ns]
 9   nb_payment_sequential          112647 non-null  float64       
 10  sum_payment_installments       112647 non-null  float64       
dtype

## Reviews

In [7]:
group_reviews = raw_reviews.groupby("order_id").agg({
    "review_id": "count",
    "review_score": "mean"})

order_items = pd.merge(order_items, group_reviews,
                       how="left",
                       on="order_id")

order_items = order_items.rename(columns={
    "review_id": "is_reviewed"})

order_items.head(3)

,order_id,order_item_id,product_id,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,nb_payment_sequential,sum_payment_installments,is_reviewed,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,1.0,2.0,1.0,5.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,1.0,3.0,1.0,4.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.0,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,1.0,5.0,1.0,5.0


In [8]:
order_items["is_reviewed"] = np.where(order_items["is_reviewed"] >= 1,
                                      True, False)

In [9]:
order_items = pd.merge(order_items, raw_customer_csv,
                       how="left",
                       on="customer_id")

order_items.head(3)

,order_id,order_item_id,product_id,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,nb_payment_sequential,sum_payment_installments,is_reviewed,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,1.0,2.0,True,5.0,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.9,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,1.0,3.0,True,4.0,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.0,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,1.0,5.0,True,5.0,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG


- customer_id : Clé dans le dataset des commandes
- customer_unique_id : Identifiant unique du client
- customer_state : Etat du client

In [10]:
data = pd.merge(order_items, products,
               how="left",
               on="product_id")

In [11]:
data

,order_id,order_item_id,product_id,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,nb_payment_sequential,...,is_reviewed,review_score,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,product_name_lenght,product_description_lenght,product_photos_qty,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,1.0,...,True,5.0,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,58.0,598.0,4.0,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,1.0,...,True,4.0,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,56.0,239.0,2.0,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,1.0,...,True,5.0,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,59.0,695.0,2.0,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,1.0,...,True,4.0,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,42.0,480.0,1.0,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-03-01 16:42:31,1.0,...,True,5.0,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,59.0,409.0,1.0,garden_tools
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-05-10 22:56:40,1.0,...,True,5.0,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,43.0,1002.0,3.0,housewares
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-23 20:31:55,1.0,...,True,5.0,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR,31.0,232.0,1.0,computers_accessories
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-28 12:22:22,1.0,...,True,5.0,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP,43.0,869.0,1.0,sports_leisure
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-16 21:59:40,1.0,...,True,5.0,eb803377c9315b564bdedad672039306,13289,vinhedo,SP,56.0,1306.0,1.0,computers_accessories


## Reduce number of categories

In [12]:
data['product_category'] = np.where((data['product_category_name'].str.contains("fashio|luggage")==True),
                                    'fashion_clothing_accessories',
                           np.where((data['product_category_name'].str.contains("health|beauty|perfum")==True),
                                    'health_beauty',
                           np.where((data['product_category_name'].str.contains("toy|baby|diaper")==True),
                                     'toys_baby',
                           np.where((data['product_category_name'].str.contains("book|cd|dvd|media")==True),
                                     'books_cds_media',
                           np.where((data['product_category_name'].str.contains("grocer|food|drink")==True), 
                                     'groceries_food_drink',
                           np.where((data['product_category_name'].str.contains("phon|compu|tablet|electro|consol")==True), 
                                     'technology',
                           np.where((data['product_category_name'].str.contains("home|furnitur|garden|bath|house|applianc")==True), 
                                                                                          'home_furniture',
                           np.where((data['product_category_name'].str.contains("flow|gift|stuff")==True),
                                     'flowers_gifts',
                           np.where((data['product_category_name'].str.contains("sport")==True),
                                     'sport',
                                     'other')))))))))

In [13]:
# drop product_category_name column
data.drop("product_category_name", axis=1, inplace=True)

In [14]:
categories_customers = data.groupby(["customer_unique_id", "product_category"])\
                        .agg({"order_item_id": "count"}).unstack()

categories_customers.columns = categories_customers.columns.droplevel(0)
categories_customers.fillna(0, inplace=True)
categories_customers["total_items"] = categories_customers.sum(axis=1)

# ratio of total items
for col in categories_customers.columns:
    if (col != "total_items"):
        categories_customers[col] = (categories_customers[col]/categories_customers["total_items"])

categories_customers.reset_index(inplace=True)
categories_customers.head(3)

product_category,customer_unique_id,books_cds_media,fashion_clothing_accessories,flowers_gifts,groceries_food_drink,health_beauty,home_furniture,other,sport,technology,toys_baby,total_items
0,0000366f3b9a7992bf8c76cfdf3221e2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0000f46a3911fa3c0805444483337064,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [15]:
products_per_order = data.groupby(["customer_unique_id", "order_id"])\
                        .agg({"order_item_id": "count"})
products_per_order = products_per_order.groupby("customer_unique_id")\
                        .agg({"order_item_id": "mean"})
products_per_order

,order_item_id
customer_unique_id,
0000366f3b9a7992bf8c76cfdf3221e2,1.0
0000b849f77a49e4a4ce2b2a4ca5be3f,1.0
0000f46a3911fa3c0805444483337064,1.0
0000f6ccb0745a6a4b88665a16c9f078,1.0
0004aac84e0df4da2b147fca70cf8255,1.0
...,...
fffcf5a5ff07b0908bd4e2dbc735a684,2.0
fffea47cd6d3cc0a88bd621562a9d061,1.0
ffff371b4d645b6ecea244b27531430a,1.0


In [16]:
recurencies = data.groupby("customer_unique_id")\
                .agg({"order_purchase_timestamp": ["min","max"]})
recurencies.columns = recurencies.columns.droplevel(0)

recurencies

,min,max
customer_unique_id,,
0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,2018-05-10 10:56:27
0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,2018-05-07 11:11:27
0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,2017-03-10 21:05:03
0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,2017-10-12 20:29:41
0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,2017-11-14 19:45:42
...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08 21:00:36,2017-06-08 21:00:36
fffea47cd6d3cc0a88bd621562a9d061,2017-12-10 20:07:56,2017-12-10 20:07:56
ffff371b4d645b6ecea244b27531430a,2017-02-07 15:49:16,2017-02-07 15:49:16


In [17]:
# Max date in dataset
max_date = data["order_purchase_timestamp"].max()

# Delay from the last order
recurencies["order_total_delay"] = [(y[1] - y[0]).round('1d').days if y[1] != y[0] 
                                    else (max_date - y[0]).round('1d').days 
                                    for x,y in recurencies.iterrows()]
recurencies.drop(["min", "max"], axis=1, inplace=True)

In [32]:
recurencies

,order_total_delay
customer_unique_id,
0000366f3b9a7992bf8c76cfdf3221e2,116
0000b849f77a49e4a4ce2b2a4ca5be3f,119
0000f46a3911fa3c0805444483337064,542
0000f6ccb0745a6a4b88665a16c9f078,326
0004aac84e0df4da2b147fca70cf8255,293
...,...
fffcf5a5ff07b0908bd4e2dbc735a684,452
fffea47cd6d3cc0a88bd621562a9d061,267
ffff371b4d645b6ecea244b27531430a,573


# Regroupement des clients

In [18]:
# Group by customers with sum or mean
# RFM : Price : Sum all the expenses 
data = data.groupby("customer_unique_id")\
            .agg({"order_id": "nunique",
                  "price": "sum",
                  "freight_value": "sum",
                  "nb_payment_sequential": "mean", 
                  "sum_payment_installments": "mean", 
                  "review_score": "mean"})

# Rename columns
data = data.rename(columns={"order_id": "nb_orders",
                            "price": "total_spend",
                            "freight_value": "total_freight",
                            "nb_payment_sequential": "mean_payment_sequential",
                            "sum_payment_installments": "mean_payment_installments",
                            "review_score": "mean_review_score"})

# Merge with other calculated features
data = pd.merge(data, categories_customers,
                how="left",
                on="customer_unique_id")

data = pd.merge(data, products_per_order,
                how="left",
                on="customer_unique_id")\
        .rename(columns={"order_item_id": "mean_nb_items"})

data = pd.merge(data, recurencies,
                how="left",
                on="customer_unique_id")

data.head(3)

,customer_unique_id,nb_orders,total_spend,total_freight,mean_payment_sequential,mean_payment_installments,mean_review_score,books_cds_media,fashion_clothing_accessories,flowers_gifts,groceries_food_drink,health_beauty,home_furniture,other,sport,technology,toys_baby,total_items,mean_nb_items,order_total_delay
0,0000366f3b9a7992bf8c76cfdf3221e2,1,129.9,12.00,1.0,8.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,116
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,18.9,8.29,1.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,119
2,0000f46a3911fa3c0805444483337064,1,69.0,17.22,1.0,8.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,542


In [19]:
data["freight_ratio"] = round(data["total_freight"] / (data["total_spend"] + data["total_freight"]),2)
data["mean_price_order"] = round(data["total_spend"] / data["nb_orders"],2)
data["total_spend"] = (data["total_spend"] + data["total_freight"])
data.drop("total_freight", axis=1, inplace=True)

In [20]:
raw_customer_csv.drop("customer_id", axis=1, inplace=True)
# Select the most frequents values for each customer
customers_info = raw_customer_csv.groupby("customer_unique_id").agg(lambda x:x.value_counts().index[0])
data = pd.merge(data, customers_info,
                how="left",
                on="customer_unique_id")

In [21]:
data

,customer_unique_id,nb_orders,total_spend,mean_payment_sequential,mean_payment_installments,mean_review_score,books_cds_media,fashion_clothing_accessories,flowers_gifts,groceries_food_drink,...,technology,toys_baby,total_items,mean_nb_items,order_total_delay,freight_ratio,mean_price_order,customer_zip_code_prefix,customer_city,customer_state
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,1.0,8.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,116,0.08,129.90,7787,cajamar,SP
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,1.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,119,0.30,18.90,6053,osasco,SP
2,0000f46a3911fa3c0805444483337064,1,86.22,1.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,542,0.20,69.00,88115,sao jose,SC
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,1.0,4.0,4.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,326,0.40,25.99,66812,belem,PA
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,1.0,6.0,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,293,0.09,180.00,18040,sorocaba,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,1,2067.42,1.0,10.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,452,0.24,1570.00,55250,sanharo,PE
95416,fffea47cd6d3cc0a88bd621562a9d061,1,84.58,1.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,267,0.23,64.89,44054,feira de santana,BA
95417,ffff371b4d645b6ecea244b27531430a,1,112.46,1.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,573,0.20,89.90,78552,sinop,MT
95418,ffff5962728ec6157033ef9805bacc48,1,133.69,1.0,5.0,5.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,124,0.14,115.00,29460,bom jesus do norte,ES


In [22]:
def null_factor(df, tx_threshold=50):
    null_rate = ((df.isnull().sum() / df.shape[0])*100).sort_values(ascending=False).reset_index()
    null_rate.columns = ['Variable','Taux_de_Null']
    high_null_rate = null_rate[null_rate.Taux_de_Null >= tx_threshold]
    return high_null_rate

In [23]:
full_null_rate = null_factor(data, 0)
full_null_rate

,Variable,Taux_de_Null
0,mean_review_score,0.732551
1,mean_payment_sequential,0.001048
2,mean_payment_installments,0.001048
3,customer_unique_id,0.000000
4,technology,0.000000
5,customer_city,0.000000
6,customer_zip_code_prefix,0.000000
7,mean_price_order,0.000000
8,freight_ratio,0.000000
9,order_total_delay,0.000000


In [24]:
data.isna().sum()

customer_unique_id                0
nb_orders                         0
total_spend                       0
mean_payment_sequential           1
mean_payment_installments         1
mean_review_score               699
books_cds_media                   0
fashion_clothing_accessories      0
flowers_gifts                     0
groceries_food_drink              0
health_beauty                     0
home_furniture                    0
other                             0
sport                             0
technology                        0
toys_baby                         0
total_items                       0
mean_nb_items                     0
order_total_delay                 0
freight_ratio                     0
mean_price_order                  0
customer_zip_code_prefix          0
customer_city                     0
customer_state                    0
dtype: int64

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95420 entries, 0 to 95419
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_unique_id            95420 non-null  object 
 1   nb_orders                     95420 non-null  int64  
 2   total_spend                   95420 non-null  float64
 3   mean_payment_sequential       95419 non-null  float64
 4   mean_payment_installments     95419 non-null  float64
 5   mean_review_score             94721 non-null  float64
 6   books_cds_media               95420 non-null  float64
 7   fashion_clothing_accessories  95420 non-null  float64
 8   flowers_gifts                 95420 non-null  float64
 9   groceries_food_drink          95420 non-null  float64
 10  health_beauty                 95420 non-null  float64
 11  home_furniture                95420 non-null  float64
 12  other                         95420 non-null  float64
 13  s

In [26]:
object_data = data[['customer_unique_id', "customer_city", "customer_state"]]

In [27]:
data

,customer_unique_id,nb_orders,total_spend,mean_payment_sequential,mean_payment_installments,mean_review_score,books_cds_media,fashion_clothing_accessories,flowers_gifts,groceries_food_drink,...,technology,toys_baby,total_items,mean_nb_items,order_total_delay,freight_ratio,mean_price_order,customer_zip_code_prefix,customer_city,customer_state
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,1.0,8.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,116,0.08,129.90,7787,cajamar,SP
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,1.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,119,0.30,18.90,6053,osasco,SP
2,0000f46a3911fa3c0805444483337064,1,86.22,1.0,8.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,542,0.20,69.00,88115,sao jose,SC
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,1.0,4.0,4.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,326,0.40,25.99,66812,belem,PA
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,1.0,6.0,5.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,293,0.09,180.00,18040,sorocaba,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95415,fffcf5a5ff07b0908bd4e2dbc735a684,1,2067.42,1.0,10.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,452,0.24,1570.00,55250,sanharo,PE
95416,fffea47cd6d3cc0a88bd621562a9d061,1,84.58,1.0,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,267,0.23,64.89,44054,feira de santana,BA
95417,ffff371b4d645b6ecea244b27531430a,1,112.46,1.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,573,0.20,89.90,78552,sinop,MT
95418,ffff5962728ec6157033ef9805bacc48,1,133.69,1.0,5.0,5.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,124,0.14,115.00,29460,bom jesus do norte,ES


In [28]:
data.drop(['customer_unique_id', "customer_city", "customer_state"], axis=1, inplace=True)

In [29]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
data = pd.DataFrame(imputer.fit_transform(data), columns = data.columns)
data

,nb_orders,total_spend,mean_payment_sequential,mean_payment_installments,mean_review_score,books_cds_media,fashion_clothing_accessories,flowers_gifts,groceries_food_drink,health_beauty,...,other,sport,technology,toys_baby,total_items,mean_nb_items,order_total_delay,freight_ratio,mean_price_order,customer_zip_code_prefix
0,1.0,141.90,1.0,8.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,116.0,0.08,129.90,7787.0
1,1.0,27.19,1.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,119.0,0.30,18.90,6053.0
2,1.0,86.22,1.0,8.0,3.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,542.0,0.20,69.00,88115.0
3,1.0,43.62,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,326.0,0.40,25.99,66812.0
4,1.0,196.89,1.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,293.0,0.09,180.00,18040.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95415,1.0,2067.42,1.0,10.0,5.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,2.0,452.0,0.24,1570.00,55250.0
95416,1.0,84.58,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,267.0,0.23,64.89,44054.0
95417,1.0,112.46,1.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,573.0,0.20,89.90,78552.0
95418,1.0,133.69,1.0,5.0,5.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,124.0,0.14,115.00,29460.0


In [30]:
data = pd.merge(data, object_data, left_index=True, right_index=True)

In [31]:
data.to_csv("data/cleaned_olist_data.csv")